# 모델 구현 과정


++ 여기서 부터는 .py 파일 직접 들어가서 수정해주세요!

### - src/KoBART/`train.py` 수정
    1. mode 변수 생성 (train/test 모드 구분하기 위함)   
    2. test_step 함수 구현    
    3. test_epoch_end 함수 구현 
    4. early stopping 함수 구현
    5. main 함수 추가 수정  
    6. lr finder 사용시     
    7. 최적의 batch size 탐색 시     

---
## 1. mode 변수 생성

<img src="img/1.png" height="100px" width="700px" align="left">

- 15번째 줄과 같이 `from rouge import Rouge`작성하여 Rouge 패키지 import 하기
- `mode 변수`와 `hparams_file` 새로 생성

#### <모델 실행 시 checkpoint_path 변수와 haparmas_file 사용에 대한 설명>

- Train
     - 모델 새로 생성 : checkpoint_path 경로 입력 x, hparams_file 경로 입력 x
     - 모델 추가 학습 (**이어서 학습**)
          - checkpoint_path 경로 입력 - 이어서 학습할 checkpoint 모델 경로 입력
          - hparams_file 경로 입력 - 해당 모델의 yaml 파일 경로 입력
- Test
     - checkpoint_path 경로 입력 - 성능 평가할 모델의 checkpoint 경로 입력  
     - hparams_file 경로 입력 - 성능 평가할 모델의 yaml 파일 경로 입력

 <br>

    **=> 즉, 모델을 처음부터 학습시킬 때 제외하고는 두 변수(checkpoint_path, hparams_file) 모두 입력 필요!**
   

---
## 2,3. test_step 및 test_epoch_end 함수 구현

<img src="img/2.png" height="100px" width="600px" align="left">  

- class KoBARTConditionalGeneration(Base) 클래스 안의 <font color=blue>test_step 함수와 test_epoch_end 함수</font> 생성
- 위치는 <font color=green>validation_epoch_end 함수 밑</font>에 생성!


### > test_step 함수 코드

    def test_step(self, batch, batch_idx):
        score = {'rouge-1':{'r':0, 'p':0, 'f':0}, 
                 'rouge-2':{'r':0, 'p':0, 'f':0}, 
                 'rouge-l':{'r':0, 'p':0, 'f':0}}
        rouge = Rouge()
        
        x = batch['input_ids']
        y = batch['label']

        output = self.model.generate(x, eos_token_id=1, max_length=512, num_beams=5)
        
        
        for i in range(len(output)):

            predict = self.tokenizer.decode(output[i], skip_special_tokens=True)
            label = y[i]
            
            s = rouge.get_scores(predict, label)[0]
            
            
            score['rouge-1']['r'] += s['rouge-1']['r']
            score['rouge-1']['p'] += s['rouge-1']['p']
            score['rouge-1']['f'] += s['rouge-1']['f']
            
            score['rouge-2']['r'] += s['rouge-2']['r']
            score['rouge-2']['p'] += s['rouge-2']['p']
            score['rouge-2']['f'] += s['rouge-2']['f']
            
            score['rouge-l']['r'] += s['rouge-l']['r']
            score['rouge-l']['p'] += s['rouge-l']['p']
            score['rouge-l']['f'] += s['rouge-l']['f']
            
            
        score['rouge-1']['r'] /= len(output)
        score['rouge-1']['p'] /= len(output)
        score['rouge-1']['f'] /= len(output)
            
        score['rouge-2']['r'] /= len(output)
        score['rouge-2']['p'] /= len(output)
        score['rouge-2']['f'] /= len(output)
            
        score['rouge-l']['r'] /= len(output)
        score['rouge-l']['p'] /= len(output)
        score['rouge-l']['f'] /= len(output)
        
        #print(score)
           
        return (score)

### > test_epoch_end 함수 코드

    def test_epoch_end(self, outputs):
        score = {'rouge-1':{'r':0, 'p':0, 'f':0}, 'rouge-2':{'r':0, 'p':0, 'f':0}, 'rouge-l':{'r':0, 'p':0, 'f':0}}
        
        for s in outputs:
            score['rouge-1']['r'] += s['rouge-1']['r']
            score['rouge-1']['p'] += s['rouge-1']['p']
            score['rouge-1']['f'] += s['rouge-1']['f']
            
            score['rouge-2']['r'] += s['rouge-2']['r']
            score['rouge-2']['p'] += s['rouge-2']['p']
            score['rouge-2']['f'] += s['rouge-2']['f']
            
            score['rouge-l']['r'] += s['rouge-l']['r']
            score['rouge-l']['p'] += s['rouge-l']['p']
            score['rouge-l']['f'] += s['rouge-l']['f']
            
        
        score['rouge-1']['r'] /= len(outputs)
        score['rouge-1']['p'] /= len(outputs)
        score['rouge-1']['f'] /= len(outputs)
            
        score['rouge-2']['r'] /= len(outputs)
        score['rouge-2']['p'] /= len(outputs)
        score['rouge-2']['f'] /= len(outputs)
            
        score['rouge-l']['r'] /= len(outputs)
        score['rouge-l']['p'] /= len(outputs)
        score['rouge-l']['f'] /= len(outputs)
        
        
        df = pd.DataFrame(score)
        df.to_csv(os.path.join(args.default_root_dir, 'rouge_score.csv'))
        
        print(df)

----
## 4. earlystopping 함수 구현

#### <기존의 main 함수>

<img src="img/7.png" height="100px" width="700px" align="left">  

**1. 기존의 main 함수에서 213번째 라인 <font color=blue>model = KoBARTConditionalGeneration(args)</font> 삭제**    
**2. 기존의 main 함수에서 233번째 라인 <font color=blue>trainer.fit(model, dm)</font> 삭제**

#### <수정된 main 함수>

<img src="img/4.png" height="100px" width="700px" align="left">  

**1. early_stopping 함수 추가 (아래 코드 복붙)**

- 위치는 checkpoint_callback 변수 밑에 추가

    early_stopping = pl.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,   # metric 성능이 몇 번의 epoch가 향상 되지않을 때 학습을 멈출건지 지정
        verbose=True,
        mode='min'
    )

**2. 329번째 줄과 같이 옵션 추가**

**3. 342번째 줄과 같이 새롭게 생성한 early_stopping 함수를 pl.Trainer.from_argparse_args 함수 인자인 <font color='red'>callback 리스트 안에 추가!</font> (빨간색 네모)**

---
## 5. main 추가 수정

<img src="img/5.png" height="100px" width="900px" align="left">  

**1. 모드에 따른 모델 생성 코드 추가** (아래 코드 복붙)
- 위치는 trainer 변수 밑에 추가

    if args.mode == 'train':       
            
        if not args.checkpoint_path and not args.hparams_file:   # 체크포인트 경로나 yaml 파일 경로가 입력되지 않았을 경우
            model = KoBARTConditionalGeneration(args)            # 모델 새로 생성
        else:                                                    # 체크포인트 경로와 yaml 파일 경로 모두 입력된 경우
            with open(args.hparams_file) as f:                   
                hparams = yaml.load(f, Loader=yaml.Loader)
            
            model = KoBARTConditionalGeneration.load_from_checkpoint(args.checkpoint_path, hparams=hparams) # 해당 모델 로드
       
        
        '''
        <lr finder 사용 시>
         lr_finder = trainer.tuner.lr_find(model=model, datamodule=dm)
         lr_finder.results
         new_lr = lr_finder.suggestion() # Pick point based on plot, or get suggestion
         model.hparams.lr = new_lr
        '''

        #trainer.tune(model,dm)  # 최적의 batch size 찾을 시
        trainer.fit(model, dm)
        
        
    elif args.mode == 'test':
        
        with open(args.hparams_file) as f:
            hparams = yaml.load(f, Loader=yaml.Loader)
            
        model=KoBARTConditionalGeneration.load_from_checkpoint(args.checkpoint_path, hparams=hparams)
       

        # test (pass in the model)
        trainer.test(model=model, datamodule=dm, verbose=True)

**2. 최적의 batch size 탐색 옵션 설명**

- 2, 4, 8, 16, 32 등 batch size 늘려가며 최적의 batch size를 탐색하는 옵션
- 사진의 341번째 줄과 같이 <font color='blue'>pl.Trainer.from_argparse_args</font> 함수 인자에 `auto_scale_batch_size='power'` 추가

    trainer = pl.Trainer.from_argparse_args(args, auto_scale_batch_size='power', logger=tb_logger,
                                            callbacks=[checkpoint_callback, early_stopping, lr_logger])
       

- 옵션 추가한 후 trainer.fit(model, dm) 하기 전에 **<font color='blue'>trainer.tune(model, dm)</font> 필수적으로 실행**
- 단, 해당 기능 사용 시 lr finder 함수와 함께 실행하면 에러가 나므로, <font color='red'>lr finder 함수 코드는 주석 처리하고 진행!</font>

**3. lr finder 코드 설명**

         lr_finder = trainer.tuner.lr_find(model=model, datamodule=dm)
         lr_finder.results
         new_lr = lr_finder.suggestion() # Pick point based on plot, or get suggestion
         model.hparams.lr = new_lrㅡ

* 자동으로 최적의 학습률을 찾아주는 함수
* lr finder 관련 설명 링크 첨부
    - https://pytorch-lightning.readthedocs.io/en/latest/advanced/lr_finder.html 
    - https://github.com/davidtvs/pytorch-lr-finder 
* 단, lr finder 사용 시 trainer.tune(model, dm)과 같이 사용하면 에러가 나므로, <font color='red'>trainer.tune(model, dm) 주석처리하고 진행!</font>

### - src/KoBART/`dataset.py` 수정

<img src="img/6.png" height="100px" width="600px" align="left"> 

**1. 50번째 줄과 58번째 줄과 같이 코드 추가하기**